In [4]:
import pandas as pd
import json
import numpy as np
from readability import Readability
import seaborn as sns
import ast
import syllables
import requests
from collections import Counter
import string
from bs4 import BeautifulSoup
import readability_lxml # note this package has been manually renamed due to clash with other readability package
from readability_lxml import Document

In [7]:
data_df = pd.read_csv('data/3-text-extraction.csv')
data_df.columns = ['','Firm','Link','Text']
data_df

,,Firm,Link,Text
0,1,Whiteleys Financial Management LLP,http://www.whiteleysaccountants.co.uk/privacy-...,['http://www.whiteleysaccountants.co.uk ( Webs...
1,2,Whiteleys Financial Management LLP,http://www.whiteleysaccountants.co.uk/news-ite...,"['Despite record wage rises , real - terms pay..."
2,3,Pioneer Point Partners LLP,https://pioneerpoint.com/privacy-and-cookie-po...,['Our privacy & cookie policy Pioneer Point Pa...
3,4,Engage Wealth Management Limited,https://engagewm.co.uk/privacy-notice/,['CLICK HERE to download a PDF version of our ...
4,5,Engage Wealth Management Limited,https://policies.google.com/privacy,"['Google Privacy Policy', 'When you use our se..."
...,...,...,...,...
4722,4826,Lady Grover's Hospital Fund for Officers' Fami...,https://forceshealth.co.uk/cookie-policy/,['Forces Health Military Private Medical Insur...
4723,4827,Onyx Commodities Limited,http://www.onyxcapitalgroup.com/cookie-policy/,"['Last updated : 09-11-2022', 'Onyx Capital Gr..."
4724,4828,Baring Asset Management Limited,http://www.barings.com/en-gb/guest/content/ter...,"['Barings LLC ( “ Barings ” ) , a Delaware lim..."
4725,4830,Baring Asset Management Limited,http://www.barings.com/en-gb/guest/content/coo...,['This Cookies Notice applies to Barings LLC a...


## Average Sentence Length

In [ ]:
texts = data_df['Text']
texts = [ast.literal_eval(i) for i in texts]

In [ ]:
avg_sentence_length = []
for i in texts:
    numb_sentences = len(i)
    total_words = 0
    for j in np.arange(len(i)):
        total_words += len(i[j])
    
    if numb_sentences != 0: 
        avg_sentence_length.append(total_words / numb_sentences)
    else:
        avg_sentence_length.append(np.NaN)

In [ ]:
data_df['Average Sentence Length'] = avg_sentence_length

## # of characters on webpage

In [ ]:
links_df = pd.read_excel('data/2-urls-all.xlsx')

# uncomment and change index for running code in batches
# links = links_df.iloc[10:15]['URL']
# firms = links_df.iloc[10:15]['Firm']

links = links_df['URL']
firms = links_df['Firm']

In [ ]:
%%time
all_charactercount = []

for link in links:
    try:
        r = requests.get(link, stream=True)
        doc = Document(r.text)
        page_content = doc.summary()
        
        soup = BeautifulSoup(page_content, features="html.parser")
        
        # Find total word count for one link
        charactercount = len(page_content)
        
        all_charactercount.append(charactercount)  
    except:
        all_charactercount.append(np.NaN)

In [ ]:
data_df['Common Perc'] = np.log(all_charactercount)

## Common Words as % of total words

In [ ]:
k_words = 'https://raw.githubusercontent.com/first20hours/google-10000-english/master/google-10000-english-usa-no-swears.txt'
response = requests.get(k_words)
data = response.text
set_of_common_words = {x for x in data.split('\n')}

words = {}
for index, row in data_df.iterrows():
    text = str(row[3]).strip()
    text = text.replace('[','').replace("]","").replace("'","").strip()
    word_list = text.split(',')
    word_list = [x.strip(' ') for x in word_list]
    word_list = [x for x in word_list if ((len(x)>1 or x.isalpha()) and x is not None)]
    words[row[2]] = word_list


common_word_lists = [[y for y in x if ((y in set_of_common_words) & (syllables.estimate(y) >= 3))]for x in words.values() ]
firm_complex_words_common_count = dict(zip(words.keys(), [len(x) for x in common_word_lists]))
firm_complex_words_common_count = dict(sorted(firm_complex_words_common_count.items(), key=lambda item: item[1]))

complex_common_words_pc = {k:(firm_complex_words_common_count[k]/len(v)) for (k,v) in words.items() if len(v)!=0}

In [ ]:
data_df['Common Perc'] = complex_common_words_pc.values()

## Export

In [8]:
data_df.to_csv('data/4-readability-measures.csv')